In [ ]:
!pip install torch
!pip install emoji
!pip install tf
!pip install -q gdown

import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re
import emoji
import string
import re #regex library
# import word_tokenize & FreqDist from NLTK
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from collections import Counter
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
import pandas as pd
import tensorflow as tf
from gensim.models import Word2Vec

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D,MaxPooling1D, Dense, Flatten, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import ReduceLROnPlateau

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


###Read the Dataset

In [ ]:
from google.colab import drive
#drive.mount('/content/drive')
!gdown --id '1zTbVRzqa71gKuxHvx2Kbrgz6rQc9ikTL'
df = pd.read_csv("/content/ISOT.csv")

/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1zTbVRzqa71gKuxHvx2Kbrgz6rQc9ikTL
To: /content/ISOT.csv
100% 117M/117M [00:01<00:00, 66.8MB/s]


###Load the GloVe and Word2Vec Model

In [ ]:
# Load the GloVe embeddings from a text file
def load_glove_embeddings(file_path):
    embeddings = {}
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            coefs = np.asarray(values[1:], dtype='float32')
            embeddings[word] = coefs
    return embeddings
!gdown --id '1ghIpaPAP1b8CWQUwFBiC4ucQs1-ASAC_'
glove_embeddings_path = '/content/glove.6B.300d.txt'
glove_embeddings = load_glove_embeddings(glove_embeddings_path)

# Set the dimensions of the GloVe embeddings
glove_embedding_dim = len(next(iter(glove_embeddings.values())))



/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1ghIpaPAP1b8CWQUwFBiC4ucQs1-ASAC_
To: /content/glove.6B.300d.txt
100% 1.04G/1.04G [00:08<00:00, 122MB/s]


In [ ]:
import gensim
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
print(gensim.__version__)
# Load Pretrained Word Embeddings (Word2Vec)
!gdown --id '1SkL4YikSgBeIsEe8idl_ktlVdah1jm8e'
word_embeddings = KeyedVectors.load_word2vec_format('/content/GoogleNews-vectors-negative300.bin',binary=True)

4.3.1
/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1SkL4YikSgBeIsEe8idl_ktlVdah1jm8e
To: /content/GoogleNews-vectors-negative300.bin
100% 3.64G/3.64G [00:30<00:00, 120MB/s] 


In [ ]:
# Get the shape of word embeddings
embedding_dimension = word_embeddings.vectors.shape[1]

print("Word Embedding Dimension:", embedding_dimension)

Word Embedding Dimension: 300


###Split The Dataset

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(df['text'], df['label'], test_size=0.2 ,random_state=42)

##Implementation

In [ ]:
# Tokenize and convert text to sequences
max_sequence_length = 100  # Define the maximum length of input sequences
tokenizer = Tokenizer()
tokenizer.fit_on_texts(x_train)
sequences_train = tokenizer.texts_to_sequences(x_train)
sequences_test = tokenizer.texts_to_sequences(x_test)

# Pad sequences to a fixed length
word_index = tokenizer.word_index
num_words = len(word_index) + 1
X_padded_train = pad_sequences(sequences_train, maxlen=max_sequence_length)
X_padded_test = pad_sequences(sequences_test, maxlen=max_sequence_length)

In [ ]:
# Define the margin loss function
def margin_loss(y_true, y_pred):
    margin = K.constant(0.1)
    y_true_float = K.cast(y_true, dtype='float32')  # Convert y_true to float32
    squared_difference_pos = K.square(K.maximum(0.9 - y_pred, 0))
    squared_difference_neg = K.square(K.maximum(y_pred - 0.1, 0))
    loss_pos = K.mean(y_true_float * squared_difference_pos)
    loss_neg = K.mean((1 - y_true_float) * squared_difference_neg)
    total_loss = loss_pos + lambda_val * loss_neg
    return total_loss

In [ ]:
# Define the model architectures
embedding_dim = 300
max_sequence_length = 100
lambda_val = 0.7  # Lambda value for margin loss

In [ ]:
# Create the embedding matrix
embedding_matrix = np.zeros((num_words, glove_embedding_dim))
for word, i in word_index.items():
    embedding_vector = glove_embeddings.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

###Static

In [ ]:
# Static Model with GloVe
static_model = Sequential()
static_model.add(Embedding(num_words, embedding_dim, input_length=max_sequence_length, trainable=False, weights=[embedding_matrix]))
static_model.add(Conv1D(128, 3, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
static_model.add(MaxPooling1D(2))
static_model.add(Flatten())
static_model.add(Dense(128, activation='relu'))
static_model.add(Dense(1, activation='sigmoid'))

learning_rate = 0.001
# Set the margin loss as the loss function
static_model.compile(loss=margin_loss, optimizer=Adam(learning_rate), metrics=['accuracy'])

###Non-Static

In [ ]:
# Non-Static Model with GloVe
non_static_model = Sequential()
non_static_model.add(Embedding(num_words, embedding_dim, input_length=max_sequence_length, trainable=True, weights=[embedding_matrix]))
non_static_model.add(Conv1D(128, 3, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
non_static_model.add(MaxPooling1D(2))
non_static_model.add(Flatten())
non_static_model.add(Dense(128, activation='relu'))
non_static_model.add(Dense(1, activation='sigmoid'))

# Set the margin loss as the loss function
non_static_model.compile(loss=margin_loss, optimizer=Adam(learning_rate), metrics=['accuracy'])

###Multichannel

In [ ]:
# Multichannel Model with GloVe
multichannel_model = Sequential()
multichannel_model.add(Embedding(num_words, embedding_dim, input_length=max_sequence_length, trainable=False, weights=[embedding_matrix]))
multichannel_model.add(Conv1D(128, 3, activation='relu'))
multichannel_model.add(MaxPooling1D(2))
multichannel_model.add(Conv1D(128, 3, activation='relu'))
multichannel_model.add(MaxPooling1D(2))
multichannel_model.add(Flatten())

# Add the second channel with GloVe embeddings
multichannel_model.add(Embedding(num_words, embedding_dim, input_length=max_sequence_length, trainable=True, weights=[embedding_matrix]))
multichannel_model.add(Conv1D(128, 3, activation='relu'))
multichannel_model.add(MaxPooling1D(2))
multichannel_model.add(Flatten())

multichannel_model.add(Dense(128, activation='relu', kernel_regularizer=l2(0.01)))
multichannel_model.add(Dense(1, activation='sigmoid'))

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, min_lr=1e-7)
learning_rate = 0.001
# Set the margin loss as the loss function
multichannel_model.compile(loss=margin_loss, optimizer=Adam(learning_rate), metrics=['accuracy'])

###Training

In [ ]:
# Train the models
static_model.fit(X_padded_train, y_train, batch_size=25, epochs=10, validation_data=(X_padded_test, y_test))

Epoch 1/10
1437/1437 [==============================] - 16s 10ms/step - loss: 0.0768 - accuracy: 0.9518 - val_loss: 0.0297 - val_accuracy: 0.9713
Epoch 2/10
1437/1437 [==============================] - 15s 10ms/step - loss: 0.0281 - accuracy: 0.9687 - val_loss: 0.0285 - val_accuracy: 0.9553
Epoch 3/10
1437/1437 [==============================] - 15s 10ms/step - loss: 0.0242 - accuracy: 0.9709 - val_loss: 0.0274 - val_accuracy: 0.9687
Epoch 4/10
1437/1437 [==============================] - 15s 11ms/step - loss: 0.0228 - accuracy: 0.9734 - val_loss: 0.0254 - val_accuracy: 0.9633
Epoch 5/10
1437/1437 [==============================] - 15s 10ms/step - loss: 0.0218 - accuracy: 0.9741 - val_loss: 0.0227 - val_accuracy: 0.9657
Epoch 6/10
1437/1437 [==============================] - 15s 10ms/step - loss: 0.0203 - accuracy: 0.9743 - val_loss: 0.0193 - val_accuracy: 0.9749
Epoch 7/10
1437/1437 [==============================] - 15s 10ms/step - loss: 0.0204 - accuracy: 0.9743 - val_loss: 0.0209 -

In [ ]:
non_static_model.fit(X_padded_train, y_train, batch_size=25, epochs=10, validation_data=(X_padded_test, y_test))

Epoch 1/10
1437/1437 [==============================] - 602s 418ms/step - loss: 0.0676 - accuracy: 0.9428 - val_loss: 0.0194 - val_accuracy: 0.9841
Epoch 2/10
1437/1437 [==============================] - 602s 419ms/step - loss: 0.0110 - accuracy: 0.9897 - val_loss: 0.0113 - val_accuracy: 0.9854
Epoch 3/10
1437/1437 [==============================] - 604s 420ms/step - loss: 0.0074 - accuracy: 0.9938 - val_loss: 0.0090 - val_accuracy: 0.9892
Epoch 4/10
1437/1437 [==============================] - 616s 428ms/step - loss: 0.0058 - accuracy: 0.9958 - val_loss: 0.0097 - val_accuracy: 0.9878
Epoch 5/10
1437/1437 [==============================] - 626s 436ms/step - loss: 0.0048 - accuracy: 0.9965 - val_loss: 0.0284 - val_accuracy: 0.9379
Epoch 6/10
1437/1437 [==============================] - 627s 436ms/step - loss: 0.0040 - accuracy: 0.9977 - val_loss: 0.0073 - val_accuracy: 0.9890
Epoch 7/10
1437/1437 [==============================] - 627s 436ms/step - loss: 0.0036 - accuracy: 0.9978 - val_

In [ ]:
multichannel_model.fit(X_padded_train, y_train, batch_size=25, epochs=10, validation_data=(X_padded_test, y_test), callbacks=[reduce_lr])

Epoch 1/10


1437/1437 [==============================] - 900s 626ms/step - loss: 0.1315 - accuracy: 0.6733 - val_loss: 0.1161 - val_accuracy: 0.7012 - lr: 0.0010
Epoch 2/10
1437/1437 [==============================] - 902s 628ms/step - loss: 0.1152 - accuracy: 0.6938 - val_loss: 0.1132 - val_accuracy: 0.6993 - lr: 0.0010
Epoch 3/10
1437/1437 [==============================] - 891s 620ms/step - loss: 0.1127 - accuracy: 0.6991 - val_loss: 0.1116 - val_accuracy: 0.6986 - lr: 0.0010
Epoch 4/10
1437/1437 [==============================] - 896s 624ms/step - loss: 0.1113 - accuracy: 0.7012 - val_loss: 0.1107 - val_accuracy: 0.7001 - lr: 0.0010
Epoch 5/10
1437/1437 [==============================] - 890s 620ms/step - loss: 0.1097 - accuracy: 0.7040 - val_loss: 0.1090 - val_accuracy: 0.6981 - lr: 0.0010
Epoch 6/10
1437/1437 [==============================] - 892s 621ms/step - loss: 0.1086 - accuracy: 0.7059 - val_loss: 0.1078 - val_accuracy: 0.7091 - lr: 0.0010
Epoch 7/10
1437/1437 [=======================

###Evaluation

In [ ]:
# Evaluate the models
static_scores = static_model.evaluate(X_padded_test, y_test, verbose=0)
non_static_scores = non_static_model.evaluate(X_padded_test, y_test, verbose=0)
multichannel_scores = multichannel_model.evaluate(X_padded_test, y_test, verbose=0)

# Print the accuracy scores
print("Static Model Accuracy: %.2f%%" % (static_scores[1] * 100))
print("Non-Static Model Accuracy: %.2f%%" % (non_static_scores[1] * 100))
print("Multichannel Model Accuracy: %.2f%%" % (multichannel_scores[1] * 100))

Static Model Accuracy: 97.43%
Non-Static Model Accuracy: 98.59%
Multichannel Model Accuracy: 70.04%
